In [1]:
from google.colab import drive
drive.mount("/content/drive")

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

Mounted at /content/drive


In [2]:
# Seed ayarlama
torch.manual_seed(190401008)
# Batch size seçilir
batch_size = 16

# Veri setini yükleme
trainData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cure_the_princess_train.csv")

# Veri setini yükleme
testData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cure_the_princess_test.csv")

# Veri setini yükleme
validationData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cure_the_princess_validation.csv")

from torch.utils.data import DataLoader, TensorDataset

# TensorDataset kullanarak train, test ve validation setlerini oluşturuyoruz
train_inputs = trainData.drop('Cured', axis=1).values
train_labels = trainData['Cured'].values
train_dataset = TensorDataset(torch.tensor(train_inputs, dtype=torch.float), torch.tensor(train_labels, dtype=torch.long))

val_inputs = validationData.drop('Cured', axis=1).values
val_labels = validationData['Cured'].values
val_dataset = TensorDataset(torch.tensor(val_inputs, dtype=torch.float), torch.tensor(val_labels, dtype=torch.long))

test_inputs = testData.drop('Cured', axis=1).values
test_labels = testData['Cured'].values
test_dataset = TensorDataset(torch.tensor(test_inputs, dtype=torch.float), torch.tensor(test_labels, dtype=torch.long))


# DataLoader kullanarak train, test ve validation setlerini batch'ler halinde modele veriyoruz
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [3]:
# Define MLP class
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(input_size, hidden_size1) # 1st hidden layer
        self.hidden2 = nn.Linear(hidden_size1, hidden_size2) # 2nd hidden layer
        self.output = nn.Linear(hidden_size2, output_size) # output layer

    def forward(self, x):
        x = torch.relu(self.hidden1(x)) # Apply ReLU activation function in the 1st hidden layer
        x = torch.relu(self.hidden2(x)) # Apply ReLU activation function in the 2nd hidden layer
        x = torch.sigmoid(self.output(x)) # Apply sigmoid activation function in the output layer
        return x

input_size = len(trainData.columns) - 1 # Number of input features, excluding the target column
hidden_size1 = 100 # Number of units in the 1st hidden layer
hidden_size2 = 50 # Number of units in the 2nd hidden layer
output_size = 2 # Number of output units (for binary classification)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Use GPU if available, else use CPU

# Define the MLP model
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
# Send the model to the device (GPU or CPU)
model = model.to(device)

# Define the Cross Entropy loss function
criterion = nn.CrossEntropyLoss()

# Define the SGD optimizer
# We can adjust the learning rate to prevent overfitting
learning_rate = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [4]:
num_epochs = 50
train_loss_list = []
validation_loss_list = []
patience=5
best_val_loss = None  # define best_val_loss 
lastStop=0

for epoch in range(num_epochs):

    # Train the model
    model.train()  # train mode on
    train_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(train_loader):
        
        # Move data to device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        ### l2 regulazition ####
        l2_lambda = 0.001
        l2_reg = torch.tensor(0.)
        for param in model.parameters():
            l2_reg += torch.norm(param, p=2)

        loss += l2_lambda * l2_reg

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)        
        correct += (predicted == labels).sum().item()

        # Print loss every few iterations
        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}],  Loss: {loss.item():.4f}")
        

    train_loss_list.append(train_loss / len(train_loader.dataset))  # add list for figure
    train_loss /= len(train_loader.dataset)
    train_acc = 100 * correct / total

    # Evaluate the model on validation set every epoch
    model.eval()
    val_loss=0.0
    with torch.no_grad():
        correct = 0
        total = 0

        for inputs, labels in val_loader:
            
            # Move data to device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass and calculate accuracy
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            ### l2 regulazition ####
            l2_lambda = 0.001
            l2_reg = torch.tensor(0.)
            for param in model.parameters():
                l2_reg += torch.norm(param, p=2)


            val_loss += loss.item() * inputs.size(0)  # Collect Losses
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        validation_loss_list.append(val_loss / len(val_loader.dataset))
        val_loss/= len(val_loader.dataset) # Calculate the average by dividing the losses by the number of data
        val_acc = correct / total
        # print(f"Validation accuracy after epoch {epoch+1}: {val_acc:.4f}")

        #EarlyStopping
        val_score=val_loss

        if best_val_loss is None:
          patience_counter=0
          best_val_loss=val_score       # Keep patience length in memory
          torch.save(model.state_dict(),"checkpoint.pt")
        elif best_val_loss<val_score:     # patience counter
          patience_counter+=1
          print("Earlystopping Patience Counter",patience_counter)
          if patience_counter==patience:           
            break
        else:
          best_val_loss=val_score
          torch.save(model.state_dict(),"checkpoint.pt")        # keeps the best model
          patience_counter=0        # reset patience after we get the best model

        lastStop=epoch+1

        #print(f"Validation accuracy after epoch {epoch+1}: {val_acc:.4f}")
        #print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Earlystopping Patience Counter 1
Earlystopping Patience Counter 1
Earlystopping Patience Counter 1
Earlystopping Patience Counter 2
Earlystopping Patience Counter 3
Earlystopping Patience Counter 1
Earlystopping Patience Counter 1
Earlystopping Patience Counter 1
Earlystopping Patience Counter 2
Earlystopping Patience Counter 3
Earlystopping Patience Counter 4
Earlystopping Patience Counter 5


In [5]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

newModel = MLP(input_size, hidden_size1, hidden_size2, output_size).to(device)

# define best model path
model_path = '/content/checkpoint.pt'

# upload best model
newModel.load_state_dict(torch.load(model_path))

# Set the model in evaluation mode
newModel.eval()

# Loop over the test set and make predictions
with torch.no_grad():
    true_labels = []
    predicted_labels = []
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = newModel(inputs)
        _, predicted = torch.max(outputs.data, 1)
        true_labels += labels.cpu().numpy().tolist()
        predicted_labels += predicted.cpu().numpy().tolist()

# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.9158
Precision: 0.9109
Recall: 0.9227
F1 Score: 0.9168
